In [1]:
from tlspt.datamodules.components.numpy_dataset import NumpyDataset
from tlspt.datamodules.components.base_site import BaseSiteDataset
from tlspt.datamodules.components.octree_dataset import OctreeDataset
from tlspt.datamodules.components.merged_dataset import MergedOctreeDataset
from tlspt.transforms import TLSSampler, UniformDownsample
from tlspt.datamodules.single_datamodule import SingleDataModule

from torch.utils.data import DataLoader


from matplotlib import pyplot as plt

from loguru import logger

import sys
import numpy as np
import time

logger.remove()  # Remove the default handler
logger.add(sys.stdout, level="INFO")

1

In [2]:
site_dataset = OctreeDataset(split_file='../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                 split='train',
                                 feature_names=['red', 'green', 'blue'],
                                 features_to_normalize=['red', 'green', 'blue'],
                                 scale=1.5,
                                 transform=UniformDownsample(num_points=1024, replace="resample_as_req"),
                                 plots_keep=['SPA01'])

2024-11-23 22:07:49.938 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from ../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-23 22:07:49.954 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 14 folders in ../data/plot_octrees/allen-spain/octrees/
2024-11-23 22:07:49.962 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDataset(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 14 plots for 'train' out of 14 plots defined in split file
2024-11-23 22:07:49.964 | INFO     | tlspt.structures.file_octree:__init__:155 - Initializing octree from <cl

In [3]:
site_dataset.octrees[0]

In [4]:
len(site_dataset.nodes)

1

In [5]:
print(f"Number of nodes in DS plot 0: {len(site_dataset.nodes[0])}")

Number of nodes in DS plot 0: 1492


In [6]:
len(site_dataset.leaf_nodes[0])  #e.g. site_dataset.leaf_nodes[number of plot][number of parent node]
num_leaves_per_node = [len(x) for x in site_dataset.leaf_nodes[0]]
print(f"Number of leaves total in DS plot 0: {sum(num_leaves_per_node)}")

Number of leaves total in DS plot 0: 1492


In [7]:
len(site_dataset)

1492

In [8]:
site_dataset.load_item(0)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [9]:
site_dataset.prepare_data(force_compute=False)

2024-11-23 22:07:57.654 | INFO     | tlspt.utils:prepare_data:95 - reading stats from ../data/plot_octrees/allen-spain/octrees/stats/stats_128a15cc4e5f0.pkl
2024-11-23 22:07:57.657 | INFO     | tlspt.utils:prepare_data:96 - for dataset OctreeDataset.{'features_to_normalize': ['red', 'green', 'blue']}.train.128a15cc4e5f0
2024-11-23 22:07:58.481 | INFO     | tlspt.utils:prepare_data:105 - mean: tensor([0.5721, 0.5507, 0.5738])
2024-11-23 22:07:58.488 | INFO     | tlspt.utils:prepare_data:106 - std: tensor([0.3137, 0.3621, 0.2952])
2024-11-23 22:07:58.489 | INFO     | tlspt.utils:prepare_data:110 - torch.float32
2024-11-23 22:07:58.490 | INFO     | tlspt.utils:prepare_data:114 - Dataset has 3 features named ['red', 'green', 'blue']. 
 Normalizing ['red', 'green', 'blue'] by mean+std. 
2024-11-23 22:07:58.492 | INFO     | tlspt.utils:prepare_data:117 - 3 channels will be zero centered and scaled to [-1,1].


In [10]:
site_dataset[9]['features'].shape

torch.Size([1024, 3])

In [11]:
site_dataset[9]['points'].shape

torch.Size([1024, 3])

In [12]:
site_dataset[0]

{'points': tensor([[-0.2019, -0.5849, -0.2222],
         [-0.1221, -0.5803, -0.1752],
         [-0.0694, -0.3904, -0.0138],
         ...,
         [ 0.3615,  0.4850,  0.0166],
         [-0.1538, -0.6101, -0.1932],
         [-0.0669,  0.5195,  0.0790]]),
 'features': tensor([[-0.6238,  1.2082, -0.4824],
         [-0.6238,  1.2082, -0.4824],
         [-0.6238,  1.2082, -0.4824],
         ...,
         [-0.8238,  0.7533, -0.7215],
         [-0.6238,  1.2082, -0.4824],
         [-0.8238,  0.7533, -0.7215]]),
 'lengths': 1024}

In [13]:
length = site_dataset[9]['lengths']
mat = np.array(site_dataset[9]['points'])  # Ensure 'mat' is a NumPy array

print(mat[0:5])  # Print first 5 points
print()
print(mat[length:length+5])  # Print points from 'length' to 'length + 5'

# Find indices of rows in 'mat' that are equal to the first row
indices = np.where(np.all(mat == mat[0], axis=1))[0]
print("Indices of rows equal to the first row:", indices)

print(len(mat))

[[ 0.2754038  -0.08090527 -0.68441606]
 [-0.13626531 -0.02692082  0.04348598]
 [-0.17056598 -0.17477037 -0.28718495]
 [-0.14936657 -0.19954045 -0.26801562]
 [ 0.11243677  0.0703957  -0.1940007 ]]

[[ 0.2754038  -0.08090527 -0.68441606]
 [-0.13626531 -0.02692082  0.04348598]
 [-0.17056598 -0.17477037 -0.28718495]
 [-0.14936657 -0.19954045 -0.26801562]
 [ 0.11243677  0.0703957  -0.1940007 ]]
Indices of rows equal to the first row: [  0 545]
1024


In [14]:
dataloader = DataLoader(site_dataset, batch_size=128, shuffle=True)

In [15]:
batch = next(iter(dataloader))

In [16]:
batch['lengths']

tensor([1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024,  915, 1024, 1024, 1024, 1024, 1024, 1024,  907, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024,  572, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024,  769, 1024, 1024,
        1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024])

In [17]:
merged_dataset = MergedOctreeDataset(
    split_files=['/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                 '/home/matt/work/tlsPT/data/plot_octrees/allen-poland/octrees/allen-poland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                 '/home/matt/work/tlsPT/data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv'],
    split='train',
    scales=[1.5, 1.5, 1.5],
    feature_names=None,
    normalize=True,
    transform=UniformDownsample(num_points=1024, replace="resample_as_req")
)

2024-11-23 22:08:00.744 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-23 22:08:01.251 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 12 folders in /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/
2024-11-23 22:08:01.255 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDataset(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 12 plots for 'train' out of 12 plots defined in split f

In [18]:
merged_dataloader = DataLoader(merged_dataset, batch_size=128, shuffle=True, num_workers=0)

In [19]:
%%time 
for i, batch in enumerate(merged_dataloader):
    if i > 5:
        break
    pass

FileNotFoundError: [Errno 2] No such file or directory: '/home/matt/work/tlsPT/data/plot_octrees/allen-poland/octrees/POL21_2021-08-25.PROJ_1,5m/voxels/661034b0b761473c.ply'

In [20]:
from tlspt.transforms import HDF5ChunkSampler

In [21]:
sampler = HDF5ChunkSampler(8192)

In [22]:
idx = sampler(1024, 53111)
for x in idx:
    print(x, x[1]-x[0])

print(sum([x[1]-x[0] for x in idx]))

(0, 1024) 1024
(2048, 3072) 1024
(7168, 8192) 1024
(15360, 16384) 1024
(19456, 20480) 1024
(26624, 27648) 1024
(41984, 44032) 2048
8192


In [23]:
from tlspt.datamodules.components.octree_dataset import OctreeDatasetHdf5
from tlspt.transforms import Padder

import importlib

In [24]:
site_dataset_hdf5 = OctreeDatasetHdf5(split_file='../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                                 split='train',
                                 feature_names=['red', 'green', 'blue', 'scalar_pathlength', 'scalar_distance', 'scalar_label'],
                                 features_to_normalize=['red', 'green', 'blue'],
                                 scale=1.5,
                                 idx_sampler=HDF5ChunkSampler(8192),
                                 transform=Padder(num_points=8192),
                                 plots_keep=['SPA01'])

2024-11-23 22:08:18.548 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from ../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-23 22:08:18.552 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 14 folders in ../data/plot_octrees/allen-spain/octrees/
2024-11-23 22:08:18.553 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDatasetHdf5(../data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 14 plots for 'train' out of 14 plots defined in split file
2024-11-23 22:08:18.554 | INFO     | tlspt.structures.file_octree:__init__:155 - Initializing oct

In [25]:
p, f = site_dataset_hdf5.load_item(7)
print(p.shape, f.shape)

torch.Size([8192, 3]) torch.Size([8192, 6])


In [26]:
site_dataset_hdf5.prepare_data(force_compute=False)

2024-11-23 22:08:18.791 | INFO     | tlspt.utils:prepare_data:244 - reading stats from ../data/plot_octrees/allen-spain/octrees/stats/stats_333b10a0a3137.pkl
2024-11-23 22:08:18.793 | INFO     | tlspt.utils:prepare_data:245 - for dataset OctreeDatasetHdf5.{'features_to_normalize': ['red', 'green', 'blue']}.train.333b10a0a3137
2024-11-23 22:08:18.818 | INFO     | tlspt.utils:prepare_data:254 - mean: tensor([0.5659, 0.5654, 0.5750,    nan,    nan,    nan])
2024-11-23 22:08:18.820 | INFO     | tlspt.utils:prepare_data:255 - std: tensor([0.3076, 0.3576, 0.2889,    nan,    nan,    nan])
2024-11-23 22:08:18.821 | INFO     | tlspt.utils:prepare_data:259 - torch.float32
2024-11-23 22:08:18.821 | INFO     | tlspt.utils:prepare_data:263 - Dataset has 6 features named ['red', 'green', 'blue', 'scalar_pathlength', 'scalar_distance', 'scalar_label']. 
 Normalizing ['red', 'green', 'blue'] by mean+std. 
2024-11-23 22:08:18.822 | INFO     | tlspt.utils:prepare_data:266 - 3 channels will be zero cente

In [27]:
print(site_dataset_hdf5[1]['points'].shape)
print(site_dataset_hdf5[1]['features'].shape)
print(site_dataset_hdf5[1]['lengths'])

torch.Size([8192, 3])
torch.Size([8192, 6])
3864


In [28]:
# #3D scatter plot 
# fig = plt.figure(figsize=(10, 10))

# ax = fig.add_subplot(111, projection='3d')
# i=np.random.randint(0, len(site_dataset_hdf5))
# ax.scatter(site_dataset_hdf5[i]['points'][:, 0], site_dataset_hdf5[i]['points'][:, 1], site_dataset_hdf5[i]['points'][:, 2], c=site_dataset_hdf5[i]['features'][:, -1], cmap='viridis', alpha=1, s=5)

In [29]:
merged_dataset_hdf5 = MergedOctreeDataset(
    split_files=['/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                 '/home/matt/work/tlsPT/data/plot_octrees/allen-poland/octrees/allen-poland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv',
                 '/home/matt/work/tlsPT/data/plot_octrees/allen-spain/octrees/allen-spain-plot_splits-tr0.7-val0.15-te0.15_seed0.csv'],
    split='train',
    scales=[1.5, 1.5, 1.5],
    feature_names=None,
    normalize=True,
    hdf5=True,
    idx_sampler=HDF5ChunkSampler(8192),
    transform=Padder(num_points=8192)
)

2024-11-23 22:08:18.858 | INFO     | tlspt.datamodules.components.base_site:__init__:49 - OctreeDatasetHdf5(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): reading splits from /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv
2024-11-23 22:08:18.860 | INFO     | tlspt.datamodules.components.base_site:__init__:56 - OctreeDatasetHdf5(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): looking for 12 folders in /home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/
2024-11-23 22:08:18.861 | INFO     | tlspt.datamodules.components.base_site:__init__:64 - OctreeDatasetHdf5(/home/matt/work/tlsPT/data/plot_octrees/allen-finland/octrees/allen-finland-plot_splits-tr0.7-val0.15-te0.15_seed0.csv, train, 1.5): found 12 plots for 'train' out of 12 plots define

In [30]:
merged_dataloader_hdf5 = DataLoader(merged_dataset_hdf5, batch_size=128, shuffle=True, num_workers=0)

In [33]:
%%time 
for i, batch in enumerate(merged_dataloader):
    if i > 10:
        break
    pass

FileNotFoundError: [Errno 2] No such file or directory: '/home/matt/work/tlsPT/data/plot_octrees/allen-poland/octrees/POL31_2021-08-18.PROJ_1,5m/voxels/66e92b65d545945f.ply'

In [32]:
%%time 
for i, batch in enumerate(merged_dataloader_hdf5):
    if i > 10:
        break
    pass

CPU times: user 5min 5s, sys: 18.4 s, total: 5min 23s
Wall time: 1min 17s
